# 1. DEFINE

#### 1.1 Define the problem

In [2]:
# This problem aims at analyzing which group/category of people had higher chance of survial in the Titanic tragedy.
# With the given set of features/columns, the problem aims at establishing the factors that contributing in 
# deciding the fate of passengers.

# 2. DISCOVER

In [3]:
# importing libraries
import numpy as np
import pandas as pd

### 2.1 Obtain dataset

In [17]:
# Training dataset
train_data = pd.read_csv('train.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
# Testing dataset
test_data = pd.read_csv('test.csv')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [19]:
# A point to be noted here is: the number of columns in train_data and test_data datasets are different. This is because the 
# train_data dataset consists of one extra column, namely 'Survived', which will be considered as the class/label for this
# dataset as per the problem definition.

# The return type of pd.read_csv() is a DataFrame. So, 'train_data' and 'test_data' are both dataframes.

In [20]:
# The dimension of the train_data dataset can be obtained as:
train_data.shape

(891, 12)

In [10]:
# This implies there are 891 entries/rows and a total of 12 columns in the dataset

In [21]:
# The dimension of test_data dataset can be obtained as:
test_data.shape

(418, 11)

In [12]:
# This implies that there are 418 entries/rows and a total of 11 columns in the dataset

Now we have two datasets. <br>
The 'train_data' consists of 891 entries and 12 columns (including the class label column 'Survived') <br>
The 'test_data' consists of 418 entries and 11 columns

### 2.2 Cleaning dataset

In order to preserve the original datasets we will create two replicas of original dataset and make changes in those replicas instead of the original dataset.

In [22]:
train = train_data
test = test_data

#### 2.2.1 First let's take a look at the columns in the datasets which do not contribute in our analysis and can be __dropped__.

In [23]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
# The columns 'PassengerId' and 'Name' does not hold relevance in deciding the survival of a passenger and hence can be dropped.
to_drop = ['PassengerId','Name']
train.drop(to_drop,axis = 1, inplace = True) 
# 'axis = 1' implies columns; 'inplace = True' implies that the change be made directly in the object 
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S


In [28]:
# To maintain consistency in the datasets, the same process will be repeated for the test dataset
test.drop(to_drop, axis = 1, inplace = True)
test.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,34.5,0,0,330911,7.8292,NaN,Q
1,3,female,47.0,1,0,363272,7.0000,NaN,S
2,2,male,62.0,0,0,240276,9.6875,NaN,Q
3,3,male,27.0,0,0,315154,8.6625,NaN,S
4,3,female,22.0,1,1,3101298,12.2875,NaN,S


In [15]:
# First we check for missing values in the dataset
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [16]:
# Out of 12 columns, 3 columns consists of missing data; of which the column 'Cabin' consists of maximum number of null values

# REFERENCES:
https://realpython.com/python-data-cleaning-numpy-pandas/
